In [ ]:
from rapidocr_onnxruntime import RapidOCR
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

if os.getcwd().endswith("tests"):
    os.chdir("..")


In [ ]:
rapidocr_reader = RapidOCR()


In [ ]:
from typing import Tuple, List, Union, Any
def normalize_rapidocr_result(result: Tuple[List[List[Union[List[List[float]], str, float]]], Any]) -> List[Tuple[List[List[float]], str, float]]:
    """Normalize RapidOCR result to match EasyOCR format."""
    if result[0] is None:
        return []
    # Extract the first element which contains the detections
    detections = result[0]
    # Convert each detection to match EasyOCR format
    normalized = []
    for detection in detections:
        # detection is [bbox, text, confidence]
        bbox = detection[0]  # Already in the correct format
        text = detection[1]  # String
        confidence = detection[2]  # Float
        normalized.append((bbox, text, confidence))
    return normalized

def process_images(folder_path, verbose = False) -> list[Any]:

    imgdata = [[open(os.path.join(folder_path, img_path), "rb").read(), print(f"Loading image {img_path}...") if verbose else ""][0] for img_path in sorted(os.listdir(folder_path)) if img_path.endswith(".png") or img_path.endswith(".jpg") or img_path.endswith(".jpeg")]
    images_loaded = [cv2.imdecode(np.frombuffer(data, np.uint8), cv2.IMREAD_COLOR) for data in imgdata]
    
    images_text = []
    for i, img in enumerate(images_loaded):
        images_text.append(normalize_rapidocr_result(rapidocr_reader(img)))
        if verbose: 
            print(f"Processed image #{i} with {len(images_text[-1])} text boxes.")
    
    return images_text


In [ ]:
report_dir = "./images/test_battle_report/"
text_in_images = process_images(report_dir, verbose = True)


In [ ]:
for entry in text_in_images[0]:
    print(entry[-2:]) # prints text and confidence only


In [ ]:
def load_images(folder_path: str, verbose = False) -> list[np.ndarray]:
    imgdata = [[open(os.path.join(folder_path, img_path), "rb").read(), print(f"Loading image {img_path}...") if verbose else ""][0] for img_path in sorted(os.listdir(folder_path)) if img_path.endswith(".png") or img_path.endswith(".jpg") or img_path.endswith(".jpeg")]
    images_loaded = [cv2.imdecode(np.frombuffer(data, np.uint8), cv2.IMREAD_COLOR) for data in imgdata]
    return images_loaded


In [ ]:
# Locate if a string is in the image
def str_in_image_from_images_list(text_in_images: list[list[str]], text_to_find: str) -> tuple[int, list]:
    """
    Processes a list of images to find the one containing the "Stat" text.
    Returns the first image that contains the text "Stat".
    """
    text_to_find = text_to_find.lower()
    for i, image_text in enumerate(text_in_images):
        for found_string in image_text:
            if isinstance(found_string, tuple):
                found_string = found_string[1]
            found_string = str(found_string).lower()
            if found_string.__contains__(text_to_find):
                print(f"Found '{text_to_find}' in image # {i}")
                return i, image_text
    raise ValueError(f"{text_to_find} page not found.")


In [ ]:

text_only = [[entry[1] for entry in all_text] for all_text in text_in_images]
resp = str_in_image_from_images_list(text_only, "Battle Overview")


In [ ]:
from src.utils import handle_split_boxes

images = load_images(report_dir)
string_to_find = "Battle Overview"
# Display the image with "Stat" text
img_idx, stats_image_text = str_in_image_from_images_list(text_in_images, string_to_find)
print(f"Image index with '{string_to_find}': {img_idx}" + f" and text: {stats_image_text}")

stats_image_text = handle_split_boxes(stats_image_text)

# Make matplotlib frame larger
plt.figure(figsize=(10, 10))

# Plot every bbox identified on the image, and print the text and confidence for each bbox
for bbox in stats_image_text:
    if isinstance(bbox, tuple):
        text, confidence = bbox[1], bbox[2]
        print(f"Text: {text}, Confidence: {confidence}")
        # Draw the bounding box on the image
        cv2.rectangle(images[img_idx], (int(bbox[0][0][0]), int(bbox[0][0][1])),
                      (int(bbox[0][2][0]), int(bbox[0][2][1])), (255, 0, 0), 2)
        # Put the text on the image
        cv2.putText(images[img_idx], text, (int(bbox[0][0][0]), int(bbox[0][0][1]) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
plt.imshow(cv2.cvtColor(images[img_idx], cv2.COLOR_BGR2RGB))
plt.axis('off')


In [ ]:
def read_outcome(text_in_image: list[str]) -> dict:
    """
    Reads the battle outcome values from the text in the image and returns a dictionary with the troop counts.
    """
    troops_count_dict = {
        "left": {
            "initial_troops": 0,
            "losses": 0,
            "injured": 0,
            "lightly_injured": 0,
            "survivors": 0
        },
        "right": {
            "initial_troops": 0,
            "losses": 0,
            "injured": 0,
            "lightly_injured": 0,
            "survivors": 0
        }
    }
    format_map = str.maketrans(
        {
            "%": "",
            "+": "",
            ",": "",
            "o": "0",
            "O": "0",
        }
    )
    
    for i, text in enumerate(text_in_image):
        if isinstance(text, tuple):
            text = text[1]
        text = str(text).strip().lower()
        if text.__contains__("troops"):
            troops_count_dict["left"]["initial_troops"] = int(str(text_in_image[i-1][1]).strip().translate(format_map))
            troops_count_dict["right"]["initial_troops"] = int(str(text_in_image[i+1][1]).strip().translate(format_map))
        elif text.__contains__("losses"):
            troops_count_dict["left"]["losses"] = int(str(text_in_image[i-1][1]).strip().translate(format_map))
            troops_count_dict["right"]["losses"] = int(str(text_in_image[i+1][1]).strip().translate(format_map))
        elif text.__contains__("injured") and not text.__contains__("lightly"):
            troops_count_dict["left"]["injured"] = int(str(text_in_image[i-1][1]).strip().translate(format_map))
            troops_count_dict["right"]["injured"] = int(str(text_in_image[i+1][1]).strip().translate(format_map))
        elif text.__contains__("lightly injured"):
            troops_count_dict["left"]["lightly_injured"] = int(str(text_in_image[i-1][1]).strip().translate(format_map))
            troops_count_dict["right"]["lightly_injured"] = int(str(text_in_image[i+1][1]).strip().translate(format_map))
        elif text.__contains__("survivors"):
            troops_count_dict["left"]["survivors"] = int(str(text_in_image[i-1][1]).strip().translate(format_map))
            troops_count_dict["right"]["survivors"] = int(str(text_in_image[i+1][1]).strip().translate(format_map))

    return troops_count_dict


In [ ]:
output = read_outcome(stats_image_text)


In [ ]:
output
